## Getting the best performance with PySpark

![Holden Karau](http://www.spark.tc/content/images/2016/06/Holden-Karau_sized.jpg)


https://www.youtube.com/watch?v=V6DkTVvy9vk <br>

https://www.youtube.com/watch?v=vfiJQ7wg81Y <br>


https://robertovitillo.com/2015/06/30/spark-best-practices/ <br>
https://www.slideshare.net/SparkSummit/getting-the-best-performance-with-pyspark

![Distrubuted Spark](http://www.bogotobogo.com/Hadoop/images/PySpark/ComponentsForDistributedExecutionInSpark.png)

![PySpark Python](https://www.packtpub.com/graphics/9781786463708/graphics/B05793_03_01.jpg )

## Word Count Example
<br>

<font color=  2e5f54 size=4 face="verdana">Spark’s simplicity makes it all too easy to ignore its execution model and still manage to write jobs that eventually complete. With larger datasets having an understanding of what happens under the hood becomes critical to reduce run-time and avoid out of memory errors</font>

### RDD operations are compiled into a Direct Acyclic Graph of RDD objects, where each RDD points to the parent it depends on:

![Direct Acyclic Graph of RDD objects](https://ravitillo.files.wordpress.com/2015/06/dag1.png)

## Best practices

### Spark UI

<font color=red size=4 face="verdana">Running Spark jobs without the **Spark UI** is like flying blind. The UI allows to monitor and inspect the execution of jobs. To access it remotely a SOCKS proxy is needed as the UI connects also to the worker nodes.

## Let's try: Word Count Example

### <font color=blue>Prepare for using DataFrames

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## <font color=C70039>First: Download some text dataset</font>

In [ ]:
import time
import os

from six.moves import urllib

#file_url = 'http://www.gutenberg.org/cache/epub/2000/pg2000.txt'
#file_name = '/resources/data/MSTC/cervantes.txt'

#file_url = 'https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt'
#file_name = '/resources/data/MSTC/t8.shakespeare.txt'

# NOTE that compressed files can be read as simple txt : NOTHING particular must be done!
file_url='http://ftp.sunet.se/mirror/archive/ftp.sunet.se/pub/tv+movies/imdb/producers.list.gz'
file_name = '/resources/data/MSTC/producers.list.gz'
    
if not os.path.exists(file_name):
    urllib.request.urlretrieve(file_url, file_name)

### <font color= red>...time to measure performance</font>

In [ ]:
from time import time

## <font color= 187b1a>Word count using RDD: reduceByKey? groupByKey?</font>
### without ordering the results...

In [ ]:
rdd_words = sc.textFile(file_name)

rdd_words = rdd_words.flatMap(lambda line: line.split()) 

rdd_words.cache()

In [ ]:
rdd_words.takeSample(False,10)

In [ ]:
rdd_words.count()

In [ ]:
t0 = time()

rdd_word_count = rdd_words.map(lambda word: (word, 1))\
    .reduceByKey(lambda x, y: x + y)\
    .collect()

tt = time() - t0
print("Task completed in {} seconds".format(round(tt,3)))

In [ ]:
rdd_word_count[0:5]

## <font color=  #7b1864 >Word count using DataFrames:</font>
### without ordering the results...

In [ ]:
from pyspark.sql import Row

### Create and cache a Dataframe with words

In [ ]:
df = sqlContext.read.text(file_name)

words=df.flatMap(lambda line: line.value.split())\
    .map(lambda x:Row(word=x, cnt=1)).toDF()
    
words.cache()

In [ ]:
words.limit(5).toPandas()

In [ ]:
words.count()

In [ ]:
t0 = time()

word_count=words.groupBy('word').count()\
    .collect()

tt = time() - t0
print("Task completed in {} seconds".format(round(tt,3)))
    

In [ ]:
word_count[0:5]

## <font color= 187b1a>Word count using RDD</font>
### NOW ordering the results...

In [ ]:
t0 = time()


rdd_word_count = rdd_words.map(lambda word: (word,1))\
    .reduceByKey(lambda x,y: x + y)\
    .map(lambda x: (x[1],x[0])) \
    .sortByKey(ascending=False) \
    .collect()
    
tt = time() - t0
print("Task completed in {} seconds".format(round(tt,3)))

In [ ]:
rdd_word_count[0:5]

## <font color=  #7b1864 >Word count using DataFrames:</font>
### Now ordering the results...

In [ ]:
t0 = time()

word_count=words.groupBy('word').count()\
    .orderBy('count',ascending=False).collect()

tt = time() - t0
print("Task completed in {} seconds".format(round(tt,3)))
    

In [ ]:
word_count[0:5]